In [0]:
# Get the Crypto currency data

!wget https://pythonprogramming.net/static/downloads/machine-learning-data/crypto_data.zip

--2019-03-29 14:12:25--  https://pythonprogramming.net/static/downloads/machine-learning-data/crypto_data.zip
Resolving pythonprogramming.net (pythonprogramming.net)... 104.237.143.20, 2600:3c00::f03c:91ff:fe84:176d
Connecting to pythonprogramming.net (pythonprogramming.net)|104.237.143.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5998694 (5.7M) [application/zip]
Saving to: ‘crypto_data.zip’

crypto_data.zip     100%[===================>]   5.72M  6.34MB/s    in 0.9s    

2019-03-29 14:12:26 (6.34 MB/s) - ‘crypto_data.zip’ saved [5998694/5998694]



In [0]:
!unzip crypto_data.zip

Archive:  crypto_data.zip
  inflating: crypto_data/BCH-USD.csv  
  inflating: crypto_data/BTC-USD.csv  
  inflating: crypto_data/ETH-USD.csv  
  inflating: crypto_data/LTC-USD.csv  


In [0]:
# check folder
!ls

crypto_data  crypto_data.zip  sample_data


In [0]:
# ORDER of columns are LOW, HIGH, OPEN, CLOSE, Volume

In [0]:
#code

#imports

import pandas as pd

df = pd.read_csv("crypto_data/LTC-USD.csv", names=["time", "low", "high", "open", "close", "volume" ])
df.head()

,time,low,high,open,close,volume
0,1528968660,96.580002,96.589996,96.589996,96.580002,9.647200
1,1528968720,96.449997,96.669998,96.589996,96.660004,314.387024
2,1528968780,96.470001,96.570000,96.570000,96.570000,77.129799
3,1528968840,96.449997,96.570000,96.570000,96.500000,7.216067
4,1528968900,96.279999,96.540001,96.500000,96.389999,524.539978


In [0]:
#using only close and volume for this
#So create new df using previous df

In [0]:
main_df = pd.DataFrame()
ratios = ["BTC-USD","LTC-USD","ETH-USD","BCH-USD"]

for ratio in ratios:
  dataset = f"crypto_data/{ratio}.csv"
  
  df = pd.read_csv(dataset, names=["time", "low", "high", "open", "close", "volume"])
  df.rename(columns={"close": f"{ratio}_close", "volume": f"{ratio}_volume"}, inplace=True) #rename so that all the colums have different names before join
  
  df.set_index("time", inplace=True) #this is the common index that will contain all the other curreny columns like primary key
  
  df = df[[f"{ratio}_close", f"{ratio}_volume"]] #get only those columns
#   print(df.head()) #check how it looks
  if len(main_df) == 0:
    main_df = df
  else:
    main_df = main_df.join(df)
    
    
main_df.head()
    
  
  

,BTC-USD_close,BTC-USD_volume,LTC-USD_close,LTC-USD_volume,ETH-USD_close,ETH-USD_volume,BCH-USD_close,BCH-USD_volume
time,,,,,,,,
1528968660,6489.549805,0.587100,96.580002,9.647200,NaN,NaN,871.719971,5.675361
1528968720,6487.379883,7.706374,96.660004,314.387024,486.01001,26.019083,870.859985,26.856577
1528968780,6479.410156,3.088252,96.570000,77.129799,486.00000,8.449400,870.099976,1.124300
1528968840,6479.410156,1.404100,96.500000,7.216067,485.75000,26.994646,870.789978,1.749862
1528968900,6479.979980,0.753000,96.389999,524.539978,486.00000,77.355759,870.000000,1.680500


In [0]:
# Some Sequence defaults

SEQ_LEN = 60    # use data of last 60 mins 
FUTURE_PERIOD_PREDICT = 3   # predict 3 mins into the future
RATIO_TO_PREDICT = "LTC-USD" #predict LTC for now

def classify(current, future):
  if float(future) > float(current): #if future price is high then its a BUY call
    return 1
  else:   # else dont BUY
    return 0

In [0]:
main_df['future'] = main_df[f"{RATIO_TO_PREDICT}_close"].shift(-FUTURE_PERIOD_PREDICT) #time series shift
main_df[[f"{RATIO_TO_PREDICT}_close",'future']].head()

,LTC-USD_close,future
time,,
1528968660,96.580002,96.500000
1528968720,96.660004,96.389999
1528968780,96.570000,96.519997
1528968840,96.500000,96.440002
1528968900,96.389999,96.470001


In [0]:
main_df['target'] = list(map(classify,main_df[f"{RATIO_TO_PREDICT}_close"], main_df["future"] ))

main_df[[f"{RATIO_TO_PREDICT}_close",'future', 'target']].head(10)

,LTC-USD_close,future,target
time,,,
1528968660,96.580002,96.500000,0
1528968720,96.660004,96.389999,0
1528968780,96.570000,96.519997,0
1528968840,96.500000,96.440002,0
1528968900,96.389999,96.470001,1
1528968960,96.519997,96.400002,0
1528969020,96.440002,96.400002,0
1528969080,96.470001,96.400002,0
1528969140,96.400002,96.400002,0


In [0]:
# separate out of sample data because model can overfit to outofsample data too!

#get 5 % out

times = sorted(main_df.index.values)  #ordered data

last_5pct = times[-int(0.05*len(times))]
last_5pct



1534922100

In [0]:
#Separate the data

validation_main_df = main_df[(main_df.index >=last_5pct)]

main_df = main_df[(main_df.index < last_5pct)]


In [0]:
from sklearn import preprocessing
from collections import deque
import random
import numpy as np


def preprocess_df(df):
  df = df.drop("future", 1) # no need for prediction
  for col in df.columns:
    if col != "target":
      df[col] = df[col].pct_change() # convert to percentage because each currency has different value but percent change will normalize it
      df.dropna(inplace=True)
      df[col] = preprocessing.scale(df[col].values) # scale it between 0 and 1
      
  df.dropna(inplace=True)
  
  sequential_data = []
  prev_days = deque(maxlen=SEQ_LEN) # de-que will pop up data once it reaches more than 60; uptil 60 it will hold the data
#   print(df.head()) # see the data
  for i in df.values:
    prev_days.append([n for n in i[:-1]])
    if len(prev_days) == SEQ_LEN:
      sequential_data.append([np.array(prev_days), i[-1]])
  
  random.shuffle(sequential_data)
  
  buys = []
  sells = []
  
  #distributing buy and sell equally
  for seq, target in sequential_data:
    if target == 0:
      sells.append([seq, target])
    elif target == 1:
      buys.append([seq, target])
      
      
  random.shuffle(buys)
  random.shuffle(sells)
  
  lower = min(len(buys), len(sells))
  
  buys = buys[:lower]
  sells = sells[:lower]
  
  
  sequential_data = buys+sells
  
  random.shuffle(sequential_data)
  
  
  X = []
  y = []
  
  for seq, target in sequential_data:
    X.append(seq)
    y.append(target)
    
   
  return np.array(X), y
  
# preprocess_df(main_df)
  
        

In [0]:
train_x, train_y = preprocess_df(main_df)
validation_x, validation_y = preprocess_df(validation_main_df)

In [0]:
print(f"train_data: {len(train_x)} validation: {len(validation_x)}" )
print(f"Dont buys: {train_y.count(0)}, buys: {train_y.count(1)}")
print(f"Validation Dont buys: {validation_y.count(0)}, buys: {validation_y.count(1)}")

train_data: 69188 validation: 3062
Dont buys: 34594, buys: 34594
Validation Dont buys: 1531, buys: 1531


In [0]:
import time

# Note: RATIO_TO_PREDICT has to be changed to see the graph for each currency; ie. run for each currency

EPOCHS = 3
BATCH_SIZE = 64
NAME = f"{RATIO_TO_PREDICT}-{SEQ_LEN}-SEQ-{FUTURE_PERIOD_PREDICT}-PRED-{int(time.time())}" 


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, CuDNNLSTM, BatchNormalization

from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint # for tensor board


# from tensorflow.keras.callbacks import EarlyStopping

In [0]:
#run tensorboard in background

# !pip install tensorboardcolab

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2019-03-29 14:16:03--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.22.145.207, 34.206.130.40, 52.21.103.149, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.22.145.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14977695 (14M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  14.28M  1.57MB/s    in 8.1s    

2019-03-29 14:16:11 (1.76 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [14977695/14977695]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [0]:
# from tensorboardcolab import *

# tbc=TensorBoardColab()

LOG_DIR = './log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)




In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [0]:
# go to this site for visualization

! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://27cdb79f.ngrok.io


In [0]:
!mkdir models

In [0]:
!ls

crypto_data	 models  ngrok-stable-linux-amd64.zip
crypto_data.zip  ngrok	 sample_data


In [0]:
tbCallBack = TensorBoard(log_dir=f"./log/{NAME}", histogram_freq=1,
                         write_graph=True,
                         write_grads=True,
                         batch_size=BATCH_SIZE,
                         write_images=True)

In [0]:


model = Sequential()

model.add(CuDNNLSTM(128, input_shape=(train_x.shape[1:]), return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())


model.add(CuDNNLSTM(128, input_shape=(train_x.shape[1:]), return_sequences=True))
model.add(Dropout(0.1))
model.add(BatchNormalization())

model.add(CuDNNLSTM(128, input_shape=(train_x.shape[1:])))
model.add(Dropout(0.2))
model.add(BatchNormalization())


model.add(Dense(32, activation="relu"))
model.add(Dropout(0.2))


model.add(Dense(2, activation="softmax"))


opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)
model.compile(loss='sparse_categorical_crossentropy',
             optimizer=opt,
             metrics=['accuracy'])


# tensorboard = TensorBoard(log_dir=f"logs/{NAME}")
filepath = "RNN_Final-{epoch:02d}-{val_acc:.3f}"
checkpoint = ModelCheckpoint("./models/{}_test.model".format(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max'))


history = model.fit(
train_x, train_y,
    batch_size=BATCH_SIZE,
    epochs=10,
    validation_data=(validation_x, validation_y),
    callbacks=[tbCallBack, checkpoint]
)


#If error clear runtime


Train on 69188 samples, validate on 3062 samples
Epoch 1/10
69184/69188 [============================>.] - ETA: 0s - loss: 0.7169 - acc: 0.5214

InvalidArgumentError: ignored